In [11]:
import urllib
import datetime
import requests
import os
import sys
import glob
import re 
import asyncio
import nest_asyncio
import aiohttp

nest_asyncio.apply()

from PyPDF2 import PdfFileReader,PdfFileWriter
from bs4 import BeautifulSoup  
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
        
start_date = date(2019, 12, 1)
end_date = date(2019, 12, 31)
end_date = end_date  + datetime.timedelta(days=1)
print(f'Start Date is {start_date}')
print(f'End Date is {end_date}')


print ('---------Download Date List  ------')
weekdays = [6,7]
effective_date_list = []
for dt in daterange(start_date, end_date):
    if dt.isoweekday() not in weekdays:
        
        # effective_date is 'datetime.date' type, so transfer to Str
        timestampStr = dt.strftime("%Y-%m-%d")
        effective_date_list.append(timestampStr)
print(effective_date_list)
print ('------------------------------------')

# Generate first page websites list
# http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/23/node_2.htm
paper_stdaily_base = "http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/"
tmp_str='2020-01-23'

paper_fetch_list = []
for idx_date in effective_date_list:
    paper_current_url = paper_stdaily_base + idx_date[0:7] + '/' + idx_date[8:10] + '/node_2.htm'
    paper_fetch_list.append(paper_current_url)

# fetch website of eachday
async def download_one(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp: 
            return await resp.text()

async def download_all(effective_date_list, sites):
    tasks = [asyncio.create_task(download_one(site)) for site in sites]
    texts = await asyncio.gather(*tasks)
    
    # generate downloads lists without 404
    
    for date_item, text_item in zip(effective_date_list, texts):
        soup_item = BeautifulSoup(text_item,'html.parser')
        download_tag = soup_item.find_all('title')
        # http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/2020-01/23/01/KJRB2020012301.pdf
        # http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/2020-01/23/02/KJRB2020012302.pdf
        if '404' not in str(download_tag):
            print(f'{date_item} has news.')
            url_base = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/'
            # Download each page of the date
            download_pdf_name_lists = []
            for page_number in range(1,9):
                download_url = url_base + date_item[0:7] + '/' + date_item[8:10] + '/' + '{:02d}'.format(page_number) + '/'+ 'KJRB' + date_item[0:4] + date_item[5:7] + date_item[8:10] + '{:02d}'.format(page_number) + '.pdf'
                
                current_file_name = 'KJRB' + date_item[0:4] + date_item[5:7] + date_item[8:10] + '{:02d}'.format(page_number) + "_pdf.pdf"
                myfile = requests.get(download_url, allow_redirects=True)
                open(current_file_name, 'wb').write(myfile.content)
                download_pdf_name_lists.append(current_file_name)
                print(f'--- Finish downloading {current_file_name} file')
            pdf_output = PdfFileWriter()
            for infn in download_pdf_name_lists:
                #print infn
                pdf_input = PdfFileReader(open(infn, 'rb'), strict=False)
                # 获取 pdf 共用多少页
                page_count = pdf_input.getNumPages()
                #print(f'The number of pages to merge is : {page_count} page')
                for i in range(page_count):
                    pdf_output.addPage(pdf_input.getPage(i))
            summaryName = date_item[0:4] + date_item[5:7] + date_item[8:10] +'_KJRB_Summary.pdf'
            pdf_output.write(open(summaryName, 'wb'))
            print(f'|========> Finish merge {summaryName} file')
    return

tmp_str1 = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/23/node_2.htm'
tmp_str2 = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/22/node_2.htm'
#paper_fetch_list = []
#paper_fetch_list.append(tmp_str1)
#paper_fetch_list.append(tmp_str2)

#print(paper_fetch_list)
print(len(paper_fetch_list))
print(len(effective_date_list))
asyncio.run(download_all(effective_date_list, paper_fetch_list)) 

Start Date is 2019-12-01
End Date is 2020-01-01
---------Download Date List  ------
['2019-12-02', '2019-12-03', '2019-12-04', '2019-12-05', '2019-12-06', '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12', '2019-12-13', '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20', '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-30', '2019-12-31', '2020-01-01']
------------------------------------
23
23
2019-12-02 has news.
--- Finish downloading KJRB2019120201_pdf.pdf file
--- Finish downloading KJRB2019120202_pdf.pdf file
--- Finish downloading KJRB2019120203_pdf.pdf file
--- Finish downloading KJRB2019120204_pdf.pdf file
--- Finish downloading KJRB2019120205_pdf.pdf file
--- Finish downloading KJRB2019120206_pdf.pdf file
--- Finish downloading KJRB2019120207_pdf.pdf file
--- Finish downloading KJRB2019120208_pdf.pdf file


|========> Finish merge 20191202_KJRB_Summary.pdf file
2019-12-03 has news.
--- Finish downloading KJRB2019120301_pdf.pdf file
--- Finish downloading KJRB2019120302_pdf.pdf file
--- Finish downloading KJRB2019120303_pdf.pdf file
--- Finish downloading KJRB2019120304_pdf.pdf file
--- Finish downloading KJRB2019120305_pdf.pdf file
--- Finish downloading KJRB2019120306_pdf.pdf file
--- Finish downloading KJRB2019120307_pdf.pdf file
--- Finish downloading KJRB2019120308_pdf.pdf file


|========> Finish merge 20191203_KJRB_Summary.pdf file
2019-12-04 has news.
--- Finish downloading KJRB2019120401_pdf.pdf file
--- Finish downloading KJRB2019120402_pdf.pdf file
--- Finish downloading KJRB2019120403_pdf.pdf file
--- Finish downloading KJRB2019120404_pdf.pdf file
--- Finish downloading KJRB2019120405_pdf.pdf file
--- Finish downloading KJRB2019120406_pdf.pdf file
--- Finish downloading KJRB2019120407_pdf.pdf file
--- Finish downloading KJRB2019120408_pdf.pdf file


|========> Finish merge 20191204_KJRB_Summary.pdf file
2019-12-05 has news.
--- Finish downloading KJRB2019120501_pdf.pdf file
--- Finish downloading KJRB2019120502_pdf.pdf file
--- Finish downloading KJRB2019120503_pdf.pdf file
--- Finish downloading KJRB2019120504_pdf.pdf file
--- Finish downloading KJRB2019120505_pdf.pdf file
--- Finish downloading KJRB2019120506_pdf.pdf file
--- Finish downloading KJRB2019120507_pdf.pdf file
--- Finish downloading KJRB2019120508_pdf.pdf file


|========> Finish merge 20191205_KJRB_Summary.pdf file
2019-12-06 has news.
--- Finish downloading KJRB2019120601_pdf.pdf file
--- Finish downloading KJRB2019120602_pdf.pdf file
--- Finish downloading KJRB2019120603_pdf.pdf file
--- Finish downloading KJRB2019120604_pdf.pdf file
--- Finish downloading KJRB2019120605_pdf.pdf file
--- Finish downloading KJRB2019120606_pdf.pdf file
--- Finish downloading KJRB2019120607_pdf.pdf file
--- Finish downloading KJRB2019120608_pdf.pdf file


...

|========> Finish merge 20191225_KJRB_Summary.pdf file
2019-12-26 has news.
--- Finish downloading KJRB2019122601_pdf.pdf file
--- Finish downloading KJRB2019122602_pdf.pdf file
--- Finish downloading KJRB2019122603_pdf.pdf file
--- Finish downloading KJRB2019122604_pdf.pdf file
--- Finish downloading KJRB2019122605_pdf.pdf file
--- Finish downloading KJRB2019122606_pdf.pdf file
--- Finish downloading KJRB2019122607_pdf.pdf file
--- Finish downloading KJRB2019122608_pdf.pdf file


|========> Finish merge 20191226_KJRB_Summary.pdf file
2019-12-27 has news.
--- Finish downloading KJRB2019122701_pdf.pdf file
--- Finish downloading KJRB2019122702_pdf.pdf file
--- Finish downloading KJRB2019122703_pdf.pdf file
--- Finish downloading KJRB2019122704_pdf.pdf file
--- Finish downloading KJRB2019122705_pdf.pdf file
--- Finish downloading KJRB2019122706_pdf.pdf file
--- Finish downloading KJRB2019122707_pdf.pdf file
--- Finish downloading KJRB2019122708_pdf.pdf file


|========> Finish merge 20191227_KJRB_Summary.pdf file
2019-12-30 has news.
--- Finish downloading KJRB2019123001_pdf.pdf file
--- Finish downloading KJRB2019123002_pdf.pdf file
--- Finish downloading KJRB2019123003_pdf.pdf file
--- Finish downloading KJRB2019123004_pdf.pdf file
--- Finish downloading KJRB2019123005_pdf.pdf file
--- Finish downloading KJRB2019123006_pdf.pdf file
--- Finish downloading KJRB2019123007_pdf.pdf file
--- Finish downloading KJRB2019123008_pdf.pdf file


|========> Finish merge 20191230_KJRB_Summary.pdf file
2019-12-31 has news.
--- Finish downloading KJRB2019123101_pdf.pdf file
--- Finish downloading KJRB2019123102_pdf.pdf file
--- Finish downloading KJRB2019123103_pdf.pdf file
--- Finish downloading KJRB2019123104_pdf.pdf file
--- Finish downloading KJRB2019123105_pdf.pdf file
--- Finish downloading KJRB2019123106_pdf.pdf file
--- Finish downloading KJRB2019123107_pdf.pdf file
--- Finish downloading KJRB2019123108_pdf.pdf file


|========> Finish merge 20191231_KJRB_Summary.pdf file
